# OTT 서비스 추천(장르 가중치 방식)

In [1]:
import pandas as pd
import numpy as np
import re
import urllib.request
import warnings
warnings.filterwarnings('ignore')

# 데이터 전처리

In [2]:
#데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/world970511/elice_OTT/master/dataset/kinolight_all_data.csv",
                        filename="OTT.csv")
data = pd.read_csv('OTT.csv',encoding='utf-8')
data.head()

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
0,고요의 바다(The Silent Sea),2021.0,TV,15세이상관람가,"['스릴러', 'SF']",['한국'],"['배두나', '공유', '이준', '김선영', '이무생', '허성태', '이성욱']",['최항용'],NaN,['넷플릭스']
1,돈 룩 업(Don't Look Up),2021.0,movie,15세이상관람가,['코미디'],['미국'],"['레오나르도 디카프리오 ', '제니퍼 로렌스', '롭 모건', '조나 힐', '마...",['아담 맥케이'],139.0,['넷플릭스']
2,설강화(Snowdrop),2021.0,TV,15세이상관람가,['드라마'],['한국'],"['정해인', '지수', '유인나', '장승조', '윤세아', '김혜윤', '정유진']",['조현탁'],NaN,['디즈니+']
3,나 홀로 집에(Home Alone),1990.0,movie,전체관람가,"['코미디', '어드벤처(모험)', '가족']",['미국'],"['맥컬리 컬킨 ', '조 페시', '다니엘 스턴 ', '존 허드', '로버츠 블로...",['크리스 콜럼버스'],105.0,['디즈니+']
4,매트릭스(The Matrix),1999.0,movie,12세이상관람가,"['액션', 'SF']",['미국'],"['키아누 리브스 ', '로렌스 피시번 ', '캐리 앤 모스 ', '휴고 위빙', ...","['릴리 워쇼스키', '라나 워쇼스키']",136.0,"['웨이브', '티빙', '왓챠', '쿠팡플레이']"


In [3]:
platform = ['넷플릭스', '웨이브', '티빙', '쿠팡플레이', '왓챠', '디즈니+', 'seezn', '라프텔']

In [4]:
#중복제거 및 기타 제거
li=['genre','country','cast','director','provider',]

for i in li:
  data[i]=data[i].apply(lambda x:re.sub(r'\([^)]*\)', '',x.replace('[','').replace(']','').replace("'",'').replace("'",'')))
  data[i]=data[i].apply(lambda x:re.sub(r' ', '',x))
  data[i]=data[i].apply(lambda x: ' '.join(set(x.strip().split(','))))

data.head(5)

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
0,고요의 바다(The Silent Sea),2021.0,TV,15세이상관람가,스릴러 SF,한국,김선영 이무생 이성욱 배두나 이준 허성태 공유,최항용,NaN,넷플릭스
1,돈 룩 업(Don't Look Up),2021.0,movie,15세이상관람가,코미디,미국,조나힐 레오나르도디카프리오 티모시샬라메 제니퍼로렌스 타일러페리 마크라이런스 롭모건,아담맥케이,139.0,넷플릭스
2,설강화(Snowdrop),2021.0,TV,15세이상관람가,드라마,한국,윤세아 정해인 지수 장승조 정유진 김혜윤 유인나,조현탁,NaN,디즈니+
3,나 홀로 집에(Home Alone),1990.0,movie,전체관람가,코미디 어드벤처 가족,미국,존허드 캐서린오하라 조페시 안젤라고덜즈 맥컬리컬킨 로버츠블로썸 다니엘스턴,크리스콜럼버스,105.0,디즈니+
4,매트릭스(The Matrix),1999.0,movie,12세이상관람가,액션 SF,미국,로렌스피시번 조판토리아노 키아누리브스 캐리앤모스 휴고위빙 글로리아포스터,릴리워쇼스키 라나워쇼스키,136.0,티빙 왓챠 쿠팡플레이 웨이브


In [5]:
# 불필요한 행 제거
data= data[data['title']!='title']

# 년도 정수형으로 변경
data['year']=data['year'].astype('float').astype('Int64')

# 제작미정,시사/교양,기타(장르가 아니라 country로 가있음;;) 정리
# 제작미정인 경우 풀릴 예정인 곳과 아닌 곳이 있고 양이 너무 많아서 그냥 장르로 변경함
def r_ch(s,i):
  li=s.split(',')
  if i in li: 
    li.remove(i)
  return ','.join (li)


def add_ch(s,i):
  li=s.split(' ')
  if i not in li: li+=[i]
  return ' '.join (li)

li=['미정','시사/교양','기타']
for i in li:
    d=data.loc[data["country"].str.contains(i)]
    data=data.loc[data["country"].str.contains(i)==False]

    d['country']=d['country'].apply(lambda x:r_ch(x,i))
    d['genre']=d['genre'].apply(lambda x:add_ch(x,i))

    data=pd.concat([data,d])

data.head()

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
0,고요의 바다(The Silent Sea),2021,TV,15세이상관람가,스릴러 SF,한국,김선영 이무생 이성욱 배두나 이준 허성태 공유,최항용,NaN,넷플릭스
1,돈 룩 업(Don't Look Up),2021,movie,15세이상관람가,코미디,미국,조나힐 레오나르도디카프리오 티모시샬라메 제니퍼로렌스 타일러페리 마크라이런스 롭모건,아담맥케이,139.0,넷플릭스
2,설강화(Snowdrop),2021,TV,15세이상관람가,드라마,한국,윤세아 정해인 지수 장승조 정유진 김혜윤 유인나,조현탁,NaN,디즈니+
3,나 홀로 집에(Home Alone),1990,movie,전체관람가,코미디 어드벤처 가족,미국,존허드 캐서린오하라 조페시 안젤라고덜즈 맥컬리컬킨 로버츠블로썸 다니엘스턴,크리스콜럼버스,105.0,디즈니+
4,매트릭스(The Matrix),1999,movie,12세이상관람가,액션 SF,미국,로렌스피시번 조판토리아노 키아누리브스 캐리앤모스 휴고위빙 글로리아포스터,릴리워쇼스키 라나워쇼스키,136.0,티빙 왓챠 쿠팡플레이 웨이브


In [6]:
#독점 및 오리지널 표시 및 필요한 provider데이터 외에 드랍

##필요한 provider외에 나머지 드랍
remove_provider_list=['네이버시리즈온', 'GooglePlay무비', '씨네폭스','아마존프라임비디오','AppleTV+']
n_data=pd.DataFrame(columns=['title',	'year',	'kind',	'KMRB',	'genre',	'country',	'cast',	'director',	'runtime(min)',	'provider'])
for i in remove_provider_list:
  data['provider']=data['provider'].apply(lambda x: x.replace(i,''))

data['provider'].dropna()

data.head()

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
0,고요의 바다(The Silent Sea),2021,TV,15세이상관람가,스릴러 SF,한국,김선영 이무생 이성욱 배두나 이준 허성태 공유,최항용,NaN,넷플릭스
1,돈 룩 업(Don't Look Up),2021,movie,15세이상관람가,코미디,미국,조나힐 레오나르도디카프리오 티모시샬라메 제니퍼로렌스 타일러페리 마크라이런스 롭모건,아담맥케이,139.0,넷플릭스
2,설강화(Snowdrop),2021,TV,15세이상관람가,드라마,한국,윤세아 정해인 지수 장승조 정유진 김혜윤 유인나,조현탁,NaN,디즈니+
3,나 홀로 집에(Home Alone),1990,movie,전체관람가,코미디 어드벤처 가족,미국,존허드 캐서린오하라 조페시 안젤라고덜즈 맥컬리컬킨 로버츠블로썸 다니엘스턴,크리스콜럼버스,105.0,디즈니+
4,매트릭스(The Matrix),1999,movie,12세이상관람가,액션 SF,미국,로렌스피시번 조판토리아노 키아누리브스 캐리앤모스 휴고위빙 글로리아포스터,릴리워쇼스키 라나워쇼스키,136.0,티빙 왓챠 쿠팡플레이 웨이브


In [7]:
data['genre']=data['genre'].apply(lambda x: [i for i in x.split(' ')])
data['provider']=data['provider'].apply(lambda x: [i for i in x.split(' ')])
data.genre.head()

0          [스릴러, SF]
1              [코미디]
2              [드라마]
3    [코미디, 어드벤처, 가족]
4           [액션, SF]
Name: genre, dtype: object

In [17]:
# 상위 10개의 작품을 선호한다고 했을 때 장르 리스트 생성 및 언급 횟수에 따른 가중치 딕셔너리 생성

genre_pop = [x for x in data.iloc[:10].genre]
genre_list = []
for i in genre_pop:
  for j in i:
    genre_list.append(j)

genre_rank = {}
for i in genre_list:
  genre_rank[i] = genre_list.count(i)
genre_rank

{'SF': 2,
 '가족': 1,
 '드라마': 3,
 '멜로/로맨스': 1,
 '스릴러': 2,
 '애니메이션': 1,
 '액션': 2,
 '어드벤처': 3,
 '코미디': 3}

In [10]:
# 장르에 따른 포인트 계산 후 데이터 프레임에 추가

rank_point = []
# a = {'코미디':5, '가족':3, '드라마':4, '스릴러':7}

for contents_genre in data.genre:
  cnt = 0
  for cg in contents_genre:
    if cg in list(genre_rank.keys()):
      cnt += genre_rank[cg]
  rank_point.append(cnt)

data['genre_point'] = rank_point

In [11]:
# 높은 순으로 정렬된 데이터 생성, 최대 포인트-4 이상의 포인트를 가진 컨텐츠만 추출

ranking = data.sort_values('genre_point', ascending=False)
ranking = ranking[ranking['genre_point']>=max(rank_point)-4]

In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

In [13]:
# 컨텐츠 인기순으로 재배열
ranking.sort_index().head()

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider,genre_point
240,월-E(WalL-E),2008,movie,전체관람가,"[코미디, 멜로/로맨스, 애니메이션, SF, 어드벤처, 가족]",미국,프레드윌러드 맥클린톨크 벤버트 캐시나지미 엘리사나이트 존라첸버거 제프갈린,앤드류스탠튼,104.0,[디즈니+],11
630,괴물(The Host),2006,movie,12세이상관람가,"[코미디, 액션, 판타지, 드라마, 스릴러, SF, 어드벤처]",한국,변희봉 송강호 오달수 이재응 배두나 박해일 고아성,봉준호,119.0,"[seezn, 넷플릭스, 쿠팡플레이, 티빙, 왓챠]",15
867,엑스맨: 퍼스트 클래스(X-Men: First Class),2011,movie,12세이상관람가,"[액션, 드라마, 스릴러, SF, 어드벤처]",미국,베스고다드 케빈베이컨 마이클패스벤더 제니퍼로렌스 로즈번 재뉴어리존스 제임스맥어보이,매튜본,132.0,[디즈니+],12
1302,곰돌이 푸 다시 만나 행복해(Christopher Robin),2018,movie,전체관람가,"[코미디, 드라마, 뮤지컬, 애니메이션, 어드벤처, 가족]",미국,브래드거렛 이완맥그리거 마크게티스 헤일리앳웰 소피오코네도 짐커밍스 닉모하메드,마크포스터,104.0,[디즈니+],11
1624,월드 워 Z(World War Z),2013,movie,15세이상관람가,"[액션, 드라마, 스릴러, SF, 어드벤처]",미국,브래드피트 다니엘라케르테스 파나모코에나 루디보우큰 매튜폭스 제임스뱃지데일 미레유에노스,마크포스터,115,"[티빙, 왓챠]",12


In [14]:
ranking.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 630 to 13708
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         30 non-null     object
 1   year          30 non-null     Int64 
 2   kind          30 non-null     object
 3   KMRB          29 non-null     object
 4   genre         30 non-null     object
 5   country       30 non-null     object
 6   cast          30 non-null     object
 7   director      30 non-null     object
 8   runtime(min)  22 non-null     object
 9   provider      30 non-null     object
 10  genre_point   30 non-null     int64 
dtypes: Int64(1), int64(1), object(9)
memory usage: 2.8+ KB


In [56]:
# 가장 많은 컨텐츠를 제공하는 플랫폼 순위
platform_pop = [x for x in ranking.provider]
platform_list = []
for i in platform_pop:
  for j in i:
    platform_list.append(j)

rank = {}
for i in platform:
  rank[i] = platform_list.count(i)
rank = sorted(rank.items(), key=lambda x:x[1], reverse=True)
print('1등 플랫폼: {}, 2등 플랫폼: {}, 3등 플랫폼: {}'.format(rank[0][0], rank[1][0], rank[2][0]))

1등 플랫폼: 넷플릭스, 2등 플랫폼: 디즈니+, 3등 플랫폼: 웨이브


In [58]:
# 가장 많은 독점/오리지널 컨텐츠를 제공하는 플랫폼 순위
original = []
for i in ranking.provider:
  if len(i) == 1:
    original.append(i[0])

original_rank = {}
for i in platform:
  original_rank[i] = original.count(i)
original_rank = sorted(original_rank.items(), key=lambda x:x[1], reverse=True)
print('1등 플랫폼: {}, 2등 플랫폼: {}, 3등 플랫폼: {}'.format(original_rank[0][0], original_rank[1][0], original_rank[2][0]))

{'seezn': 0,
 '넷플릭스': 10,
 '디즈니+': 8,
 '라프텔': 0,
 '왓챠': 0,
 '웨이브': 2,
 '쿠팡플레이': 0,
 '티빙': 0}